In [83]:
from client.api.notebook import Notebook

ok = Notebook('a3.ok')

ok.auth(inline=True, force=True)

Assignment: A3 HMM
OK, version v1.18.1


Open the following URL:

https://okpy.org/client/login/

After logging in, copy the code from the web page and paste it into the box.
Then press the "Enter" key on your keyboard.

Paste your code here: 7xW5QqiZYFBEOS309Q1Y7dzxKtqJge
Successfully logged in as SajiaZafreen@u.boisestate.edu


In [40]:
import pandas as pd
from collections import Counter as ctr
from operator import itemgetter

In [41]:
train = pd.read_csv('train.txt', delimiter=' ', names=['word', 'pos', 'other'])
train['word'] = train.word.str.lower()

In [3]:
train[:25]

,word,pos,other
0,confidence,NN,B-NP
1,in,IN,B-PP
2,the,DT,B-NP
3,pound,NN,I-NP
4,is,VBZ,B-VP
5,widely,RB,I-VP
6,expected,VBN,I-VP
7,to,TO,I-VP
8,take,VB,I-VP
9,another,DT,B-NP


In [4]:
train['ppos'] = train.pos.shift(1)
train['ppos'] = train.ppos.fillna('O')
train = train.drop('other',axis=1)

train[:10]

,word,pos,ppos
0,confidence,NN,O
1,in,IN,NN
2,the,DT,IN
3,pound,NN,DT
4,is,VBZ,NN
5,widely,RB,VBZ
6,expected,VBN,RB
7,to,TO,VBN
8,take,VB,TO
9,another,DT,VB


### Write the functions

$$P(T_i|T_{i-1})$$

$$P(W_i|T_i)$$

In [5]:
#del list
pos = list(set(train.pos))
smooth = 0.00001

len(pos), pos

(44,
 [',',
  'MD',
  'PRP',
  'TO',
  "''",
  'NNP',
  'JJ',
  'VBP',
  'VBZ',
  'CD',
  'VB',
  'NN',
  'WDT',
  '``',
  'RBS',
  'RP',
  'WRB',
  'EX',
  'WP',
  'FW',
  'VBD',
  'VBN',
  'SYM',
  ':',
  'IN',
  'NNPS',
  'UH',
  'WP$',
  'PRP$',
  'PDT',
  '.',
  '$',
  'JJR',
  'NNS',
  '#',
  ')',
  'JJS',
  'POS',
  'VBG',
  'RBR',
  '(',
  'DT',
  'CC',
  'RB'])

## Transition Probability

In [6]:
#transition
pos_ppos = {}
pos_ppos_denom = {}
for col_name in set(train.ppos):
    sub_frame = train[train.ppos == col_name] # same column together
    counted = ctr(sub_frame.pos)
    pos_ppos[col_name] = counted
    pos_ppos_denom[col_name] = len(sub_frame)
    

# conditional 
def Ptt(Ti,Tprev):
    if Ti not in pos_ppos[Tprev]: return smooth
    return pos_ppos[Tprev][Ti] / pos_ppos_denom[Tprev]

sub_frame.pos  # all follow a particular pos tag

#print (len(sub_frame))
#counted    # other tags follow a particular pos tag how many times

#Ptt('IN','RBS' )

6         VBN
141        RB
142        IN
203         ,
214       VBN
         ... 
211112    VBN
211367     JJ
211529      .
211704    VBG
211726      .
Name: pos, Length: 6607, dtype: object

## Emission Probability

In [7]:
#emission

word_pos = {}
word_pos_denom = {}
for col_name in set(train.pos):
    sub_frame = train[train.pos == col_name]
    counted = ctr(sub_frame.word)
    word_pos[col_name] = counted
    word_pos_denom[col_name] = len(sub_frame)
    

# conditional 
def Pwt(W, T):
    if W not in word_pos[T]: return smooth
    return word_pos[T][W] / word_pos_denom[T]

#print(col_name)
#counted

In [8]:
Pwt('the','DT')

0.583419689119171

In [9]:
Pwt('mr.', 'NNP')

0.03947897807282237

In [10]:
Ptt ('NNP', 'DT')

0.1110444505044996

### Evaluate: build the trellis (Example)

- example trellis with an example sequence

In [11]:
sequence = ['the', 'government']

In [12]:
emissions = []
transitions = []
for word in sequence:
    emission = [(t,Pwt(word, t)) for t in pos]
    transition = [(tprev,Ptt(ti,tprev)) for ti in pos for tprev,prob in emission]
    emissions.append(emission)
    transitions.append(transition)
    
#print (word)
#emission

In [13]:
sorted(emission, key = itemgetter(1))

[(',', 1e-05),
 ('MD', 1e-05),
 ('PRP', 1e-05),
 ('TO', 1e-05),
 ("''", 1e-05),
 ('JJ', 1e-05),
 ('VBP', 1e-05),
 ('VBZ', 1e-05),
 ('CD', 1e-05),
 ('VB', 1e-05),
 ('WDT', 1e-05),
 ('``', 1e-05),
 ('RBS', 1e-05),
 ('RP', 1e-05),
 ('WRB', 1e-05),
 ('EX', 1e-05),
 ('WP', 1e-05),
 ('FW', 1e-05),
 ('VBD', 1e-05),
 ('VBN', 1e-05),
 ('SYM', 1e-05),
 (':', 1e-05),
 ('IN', 1e-05),
 ('NNPS', 1e-05),
 ('UH', 1e-05),
 ('WP$', 1e-05),
 ('PRP$', 1e-05),
 ('PDT', 1e-05),
 ('.', 1e-05),
 ('$', 1e-05),
 ('JJR', 1e-05),
 ('NNS', 1e-05),
 ('#', 1e-05),
 (')', 1e-05),
 ('JJS', 1e-05),
 ('POS', 1e-05),
 ('VBG', 1e-05),
 ('RBR', 1e-05),
 ('(', 1e-05),
 ('DT', 1e-05),
 ('CC', 1e-05),
 ('RB', 1e-05),
 ('NNP', 0.0002011667672500503),
 ('NN', 0.005738547782532259)]

### Evaluate: decode

In [14]:
#sorted(emission, key = itemgetter(1)) # sort by 2nd element

In [15]:
hyp = []

#for e,t in zip(emissions, transitions):
for e in emissions:
    # argmax of e
    e = sorted(e, key = itemgetter(1))[-1][0]
    hyp.append(e)

### Evaluate: check accuracy

- load test data
- pass all word sequences through your hmm
- decode
- find the most probable sequence
- compare to the "gold"

In [16]:
hyp

['DT', 'NN']

# Greedy Decoder

### Loading the Test Data

In [72]:
test = pd.read_csv('test.txt', delimiter=' ', names=['word', 'pos', 'other'])
test['word'] = test.word.str.lower()

In [82]:
test

,word,pos,ppos
0,rockwell,NNP,O
1,international,NNP,NNP
2,corp.,NNP,NNP
3,'s,POS,NNP
4,tulsa,NNP,POS
...,...,...,...
47372,according,VBG,","
47373,to,TO,VBG
47374,mr.,NNP,TO
47375,harlow,NNP,NNP


In [73]:
test['ppos'] = test.pos.shift(1)
test['ppos'] = test.ppos.fillna('O')
test = test.drop('other',axis=1)

test[:30]

,word,pos,ppos
0,rockwell,NNP,O
1,international,NNP,NNP
2,corp.,NNP,NNP
3,'s,POS,NNP
4,tulsa,NNP,POS
5,unit,NN,NNP
6,said,VBD,NN
7,it,PRP,VBD
8,signed,VBD,PRP
9,a,DT,VBD


  ### Sorting the Sentence Sequence

In [74]:
from more_itertools import split_after

#test.values
test_sen_lists = []
test_word = test['word'].values.tolist()
test_sen_list = list(split_after(test_word, lambda x: x == '.'))
test_sen_list[0]

['rockwell',
 'international',
 'corp.',
 "'s",
 'tulsa',
 'unit',
 'said',
 'it',
 'signed',
 'a',
 'tentative',
 'agreement',
 'extending',
 'its',
 'contract',
 'with',
 'boeing',
 'co.',
 'to',
 'provide',
 'structural',
 'parts',
 'for',
 'boeing',
 "'s",
 '747',
 'jetliners',
 '.']

### Sorting Tag Sequence

In [75]:
test_tag_sen_lists = []
test_tag = test['pos'].values.tolist()
test_tag_sen_list = list(split_after(test_tag, lambda x: x == '.'))
test_tag_sen_list[0]

['NNP',
 'NNP',
 'NNP',
 'POS',
 'NNP',
 'NN',
 'VBD',
 'PRP',
 'VBD',
 'DT',
 'JJ',
 'NN',
 'VBG',
 'PRP$',
 'NN',
 'IN',
 'NNP',
 'NNP',
 'TO',
 'VB',
 'JJ',
 'NNS',
 'IN',
 'NNP',
 'POS',
 'CD',
 'NNS',
 '.']

### Counting Probabilty by Emission

In [62]:
emissions = []
transitions = []
hypA = []
    
def count_emission (sequen):
    for word in sequen:
        emission = [(t,Pwt(word, t)) for t in pos]
        emissions.append(emission)
        #transition = [(tprev,Ptt(ti,tprev)) for ti in pos for tprev,prob in emission]
        #transitions.append(transition)
    for e in emissions:
        # argmax of e
        e = sorted(e, key = itemgetter(1))[-1][0]
        hypA.append(e)
    return hypA
    

#for e,t in zip(emissions, transitions):
hyp_return_list = []
#for i in test_sen_list[0]:
hyp_return = count_emission(test_sen_list[0])
print(hyp_return)
    #hyp_return_list.append(hyp_return)
    

['RB', 'NNP', 'NNP', 'POS', 'NNP', 'NN', 'VBD', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'RB', 'PRP$', 'NN', 'IN', 'NNP', 'NNP', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'NNP', 'POS', 'CD', 'RB', '.']


## Testing Code (Not Part of Assignment)

In [23]:
from operator import mul
l0 = ['a','b']
l1 = [('a', 2), ('b', 3)]
l2 = [('a', 4), ('b', 9)]
l11 = [x[1] for x in l1]
l21 = [x[1] for x in l2]

print(l11, l21)
#prod = [a * b for a, b in zip(l11, l21)]
#prod = list(map(mul, l11, l21))
prod = [l11[i]*l21[i] for i in range(len(l11))]
print (prod)
[(l0[i], prod[i]) for i in range(0,len(l0))]
#list(zip(l0, prod))

[2, 3] [4, 9]
[8, 27]


[('a', 8), ('b', 27)]

In [24]:
m=[]
m0 = ('x',12)
m1 = ('y', 13)
m.append(m0)
print(m)
m.append(m1)
m
m[1][1]

[('x', 12)]


13

In [25]:
n=[]
n0 = [('a', 6), ('b', 8)]
n1 = [('a', 7), ('b', 6)]
t = [(x,y*2) for (x,y) in n0]
t

[('a', 12), ('b', 16)]

## Counting Probability with Emission and Transition

In [63]:
#del list

def count_Prob(sequen):
    return_hyp = []
    #print(len(pos))
    for word in sequen[0:1]:
        emission1 = [(t,Pwt(word, t)) for t in pos]
        max_prob1 = sorted(emission1, key = itemgetter(1))[-1]
        return_hyp.append(max_prob1)
        #print(return_hyp)
    
    # arg_max from w1
    for word in sequen[1:]:
        emission = [(t,Pwt(word, t)) for t in pos]
        ePrevMax = return_hyp[sequen.index(word)-1][0]
        transition = [(x,Ptt(x,ePrevMax)) for (x,y) in emission]
        
        
        #multiply
        tran_prob = [x[1] for x in transition]
        emis_prob = [x[1] for x in emission]
        keys = [x[0] for x in emission]
        keyes = [x[0] for x in transition]
        emi_tran = [tran_prob[i]*emis_prob[i] for i in range(len(tran_prob))]
        proba = [(keys[i],emi_tran[i]) for i in range(0,len(keys))]
        
        #current word highest probable tag
        ecurrMax = sorted(proba, key = itemgetter(1))[-1]
        #print(ecurrMax)
        
        #add to the return list
        return_hyp.append(ecurrMax)
    
    return return_hyp
        
        

In [27]:
#hyp = count_Prob(test_sen_list[0])
#print(hyp)
#probi =[x[0] for x in hyp]
#print(probi)

## Get Tags using the Train set and Compare the Tags

In [76]:
p_list =[]   # tag list of lists
for i in test_sen_list:
    hyp = count_Prob(i)
    p =[x[0] for x in hyp]
    p_list.append(p)

#counted tags
l_cal = [item for sublist in p_list for item in sublist]  

# expected tags
l_expect = test['pos'].values.tolist()   

def match_elements(list_a, list_b):
    match = []
    for x, y in zip(list_a, list_b):
        if x == y:
            match.append(1)
    return match

matches = match_elements(l_cal, l_expect)
#print(len(l_cal), len(l_expect))
print(len(matches)/len(l_cal))

0.9039407307343226


## Confusion Matrix

In [61]:
import nltk
gold = l_cal
test = l_expect
cm = nltk.ConfusionMatrix(gold, test)
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=15))

    |                           N      N                                  V                                  V |
    |      N      I      D      N      N      J                    C      B      R      V      C      T      B |
    |      N      N      T      P      S      J      ,      .      D      D      B      B      C      O      N |
----+----------------------------------------------------------------------------------------------------------+
 NN | <12.6%>  0.0%   0.0%   1.1%   0.3%   0.5%      .      .   0.0%   0.0%   0.0%   0.1%      .      .   0.0% |
 IN |   0.2% <10.6%>  0.0%   0.2%   0.1%   0.1%      .      .   0.0%   0.0%   0.2%   0.0%      .      .   0.1% |
 DT |   0.2%   0.0%  <8.4%>  0.5%   0.1%   0.3%      .      .   0.2%   0.0%   0.1%   0.0%      .      .   0.2% |
NNP |   0.6%      .      .  <7.3%>  0.2%   0.2%      .      .   0.0%   0.0%   0.0%   0.0%   0.0%      .   0.0% |
NNS |      .      .      .   0.1%  <5.6%>     .      .      .      .      .      .      .      .

### Common Errors

- Tagging NN for NNP 0.6% of the words
- Tagging NN for DT 0.2% of the words
- Tagging NNP for DT for 0.5% of the words
- Tagging NNP for jj 0.5% of the words

## Practice Testing (Not part of Assignment)

In [29]:
#print(p_list) #list of list
import numpy as np



l_cal = [item for sublist in p_list for item in sublist]
l_expect = test['posT'].values.tolist()
d = {'Cal':l_cal,'Expect':l_expect}
dp = pd.DataFrame(d)



In [78]:
count = sum( [ len(e) for e in test_sen_list])
#print(count)

In [79]:
hyp_return_list = []
for i in test_sen_list:
    hyp_return = count_emission(i)
    #print(hyp_return)
    hyp_return_list.append(hyp_return)

count = sum( [ len(e) for e in hyp_return_list])
#print(count)

In [77]:
#flat_list = [item for sublist in hyp_return_list for item in sublist]

#print(len(flat_list))
#print(len(test_tag))
#print (len(test_sen_list))

In [85]:
ok.submit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'A3-hmm.ipynb'.
Submit... 100% complete
Submission successful for user: SajiaZafreen@u.boisestate.edu
URL: https://okpy.org/bsu/nlp/sp21/a3/submissions/l7Nw4l

